다음 강의는 [DeepLearningAI의 Multi AI Agent Systems with crewAI](https://learn.deeplearning.ai/courses/multi-ai-agent-systems-with-crewai/lesson/4/create-agents-to-research-and-write-an-article-(code)) 입니다.

# L2: 연구 및 기사 작성을 위한 에이전트 생성

이제 연구 및 기사 작성을 위한 에이전트를 생성해 보겠습니다. 이 에이전트들은 주제에 대한 정보를 수집하고 분석하여 풍부하고 흥미로운 기사를 작성하는 데 도움이 될 것입니다. 에이전트들은 다양한 소스에서 정보를 수집하고 필요한 내용을 요약하여 제공할 수 있습니다. 또한 에이전트들은 특정 주제에 대한 전문 지식을 가지고 있어 정확하고 신뢰할 수 있는 정보를 제공할 수 있습니다.

에이전트들은 자연어 처리 및 기계 학습 기술을 사용하여 효율적으로 정보를 수집하고 분석할 수 있습니다. 또한 에이전트들은 사용자의 요구에 맞게 커스터마이징되어 특정 주제나 관심사에 대한 정보를 제공할 수 있습니다.

이제 연구 및 기사 작성을 위한 에이전트를 생성하여 효율적이고 효과적인 작업을 수행할 수 있도록 해보세요.안녕하세요! 무엇을 도와드릴까요?이 강의에서는 다중 에이전트 시스템의 기본 개념을 소개하고 crewAI 프레임워크에 대한 개요를 얻게 될 것입니다.

- [OPEN API key 생성 바로가기](https://platform.openai.com/api-keys)
- [GROQ API key 생성 바로가기](https://console.groq.com/keys)

다음 명령어를 사용하여 crewai, crewai_tools 및 langchain_community 패키지를 설치할 수 있습니다.

- 가상환경만들기 및 실행하기
```
conda create -n crewai python=3.10
conda activate crewai
pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29 langchain_openai ipykernel langchain python-dotenv
python -m ipykernel install --user --name=crewai
```
- 텍스트문서 하나를 열고 api키를 저장 후 .env파일을 만들어서 디렉토리에 보관하세요

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- 크루AI 라이브러리에서 가져오기.

In [2]:
from crewai import Agent, Task, Crew
from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq
from dotenv import load_dotenv
import os
load_dotenv()

llm = ChatOpenAI(model= "gpt-3.5-turbo", api_key=os.getenv("OPENAI_API_KEY"))
llm2 = ChatGroq(model="llama3-70b-8192",
               api_key = os.getenv("GROQ_API_KEY"))

- 에이전트들을 위한 LLM으로 OpenAI의 `gpt-3.5-turbo`를 사용하게 될 것입니다.

**참고 사항:** crewAI는 에이전트의 LLM으로 다른 인기있는 모델도 사용할 수 있습니다. 몇 가지 예시는 노트북의 [맨 아래](#1)에서 확인할 수 있습니다.

- CrewAI는 Agent, Task 를 각각 만들고 디테일하게 정의하여 연결 후, crew 객체에 모아서 실행하게 됩니다.
    - 각 Agent는 명확하게 자신의 역할대로, 맥락을 이해하며 업무 수행
    - 각 task는 디테일하게 요건을 명확화 하여 정의

![crew2.png](img/crew2.png)

## 에이전트 생성

에이전트는 사용자와 상호작용하고 작업을 수행하는 프로그램입니다. 에이전트를 만들 때는 몇 가지 중요한 요소를 고려해야 합니다. 에이전트의 목적과 기능을 명확히 이해하고, 사용자와의 상호작용 방식을 결정해야 합니다.

에이전트를 만들 때 고려해야 할 몇 가지 중요한 단계은 다음과 같습니다:

1. **목적 설정**: 에이전트가 수행해야 하는 작업과 목표를 명확히 정의합니다.
2. **상호작용 방식 결정**: 에이전트가 사용자와 어떻게 상호작용할지 결정합니다. 이는 대화형 인터페이스, 텍스트 기반 인터페이스, 음성 인식 등 다양한 형태로 이루어질 수 있습니다.
3. **기술 선택**: 에이전트를 구현하기 위해 어떤 기술을 사용할지 결정합니다. 이는 자연어 처리, 기계 학습, 강화 학습 등 다양한 기술을 포함할 수 있습니다.
4. **테스트 및 평가**: 에이전트를 테스트하고 성능을 평가하여 사용자 경험을 향상시키는 데 도움이 됩니다.



### Agent: Planner
참고 : 여러 문자열을 사용하는 이점은 다음과 같습니다.

<br><br>
varname = "텍스트의 1번째 줄"<br>
          "텍스트의 2번째 줄"<br>
<br>
세 개의 따옴표로 된 문서 문자열과 대조적으로
<br>

varname = """텍스트의 1번째 줄<br>
             텍스트의 2번째 줄<br>
          """<br><br>

여러 문자열을 사용하면 공백과 줄 바꿈 문자를 추가하지 않아 LLM에 전달하기에 더 나은 형식으로 만들 수 있습니다.

In [3]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic."
              "You are good at Korean language.",
    llm=llm, # 원하는 모델을 넣을 수 있습니다.
    allow_delegation=False,
    verbose=True
)

### Agent:Writer

In [4]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements."
              "You are good at Korean language.",
    llm=llm2, # 원하는 모델을 넣을 수 있습니다.
    allow_delegation=False,
    verbose=True
)

### Agent:Editor

In [5]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible."
              "You are good at Korean language.",
    llm=llm2, # 원하는 모델을 넣을 수 있습니다.
    allow_delegation=False,
    verbose=True
)

## 작업 생성

작업을 만들 때는 명확하고 구체적인 목표를 설정하는 것이 중요합니다. 작업을 생성할 때 고려해야 할 몇 가지 중요한 단계가 있습니다.

1. **목표 설정**: 작업의 목표를 명확하게 정의하고 어떤 결과물을 얻고자 하는지 결정합니다.

2. **작업 내용 정의**: 작업의 내용을 상세히 기술하여 무엇을 해야 하는지 명확하게 이해할 수 있도록 합니다.

3. **마감일 설정**: 작업을 완료해야 하는 기한을 설정하여 시간을 효율적으로 관리합니다.

4. **우선순위 설정**: 작업의 중요도에 따라 우선순위를 정하고 중요한 작업을 먼저 처리합니다.

5. **할당 및 추적**: 작업을 담당자에게 할당하고 진행 상황을 추적하여 작업이 원활하게 진행되도록 합니다.

작업을 만들 때 이러한 단계를 따르면 작업을 효율적으로 관리하고 목표를 달성하는 데 도움이 됩니다.Hello! How can I assist you today?**Task 1: Data Entry**

- **Description:** Enter customer information into the database.
- **Expected Output:** Accurate and complete customer profiles in the database.
- **Agent:** Data entry specialist.

**Task 2: Customer Support**

- **Description:** Respond to customer inquiries and provide assistance.
- **Expected Output:** Satisfied customers with their questions answered and issues resolved.
- **Agent:** Customer support representative.

**Task 3: Social Media Management**

- **Description:** Create and schedule posts on social media platforms.
- **Expected Output:** Engaging and consistent social media presence.
- **Agent:** Social media manager.

**Task 4: Sales Calls**

- **Description:** Make outbound sales calls to potential customers.
- **Expected Output:** Increased sales and customer acquisition.
- **Agent:** Sales representative.

### 작업: 계획

1. 문제를 분석하고 이해한다.
2. 해결책을 찾기 위해 가능한 방법을 고려한다.
3. 해결책을 구현하기 위한 단계별 계획을 세운다.
4. 계획을 실행하고 문제를 해결한다.
5. 결과를 평가하고 필요한 조정을 한다.
6. 성공적으로 해결된 문제에 대한 배운 점을 기록하고 다음 비슷한 문제에 대비한다.

### Task: Plan

In [6]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}."
        "2. Identify the target audience, considering "
            "their interests and pain points."
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action."
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources."
        "in Korean" ,
    agent=planner,
)

Hello, how can I help you today?

### Task: Write

In [7]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}."
        "2. Incorporate SEO keywords naturally."
        "3. Sections/Subtitles are properly named "
            "in an engaging manner."
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion."
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice."
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs."
        "in Korean",
    agent=writer,
)

I am ready to help with editing. Please provide the text that needs editing.

### Task: edit

In [8]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs."
                    "in Korean",
    agent=editor
)

## Creating the Crew

CrewAI 객체 생성 


In [9]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=2
)

## Running the Crew


**참고**: LLMs는 동일한 입력에 대해 다른 출력을 제공할 수 있으므로, 여러분이 보는 것과 다를 수 있습니다.

In [ ]:
result = crew.kickoff(inputs={"topic": "최고의 하루를 보내기 위한 5가지 아침습관"})

In [12]:
from IPython.display import Markdown
Markdown(result)

**최고의 하루를 보내기 위한 5가지 아침습관**
=====================================================

**I. 소개**
------------

아침습관의 중요성은 우리의 일상 생활에 큰 영향을 미칩니다. 최고의 하루를 보내기 위해서는 적절한 아침습관이 필수적입니다. 이 글에서는 최고의 하루를 보내기 위한 5가지 아침습관을 소개하고, 이를 통해 더 나은 삶을 살 수 있는 방법을 제안합니다.

최고의 하루를 보내기 위한 목표 설정은 아침부터 시작됩니다. 아침에 일어나는 것은 우리의 일상 생활의 시작점입니다. 따라서, 적절한 아침습관을 형성하는 것은 매우 중요합니다. 아침습관은 우리의 에너지를 높이고, 하루를 시작하는 데 필요한 동기를 부여합니다.

**II. 5가지 아침습관**
---------------------

### 1. 일어나서 첫 번째로 하는 습관

일어나서 첫 번째로 하는 습관은 아침에 일어나는 것부터 시작됩니다. 아침에 일어나는 것은 우리의 일상 생활의 시작점입니다. 따라서, 아침에 일어나는 것이 중요한 것입니다. 아침에 일어나는 것은 우리의 에너지를 높이고, 하루를 시작하는 데 필요한 동기를 부여합니다. 이를 통해 우리는 하루를 시작하는 데 필요한 에너지를 얻을 수 있습니다.

### 2. 건강한 아침식사의 중요성

건강한 아침식사는 우리의 건강을 지키는 데 중요한 역할을 합니다. 아침식사는 우리의 에너지를 충전하고, 하루를 시작하는 데 필요한 영양소를 공급합니다. 따라서, 건강한 아침식사를 하는 것은 아침습관의 중요한 부분입니다. 건강한 아침식사는 우리의 신체와 마음을 건강하게 하는 데 도움이 됩니다.

### 3. 명상 또는 운동으로 하루를 시작하는 방법

명상 또는 운동은 우리의 몸과 마음을 건강하게 하는 데 중요한 방법입니다. 아침에 명상 또는 운동을 하는 것은 우리의 스트레스를 줄이고, 하루를 시작하는 데 필요한 에너지를 얻을 수 있습니다. 이를 통해 우리는 하루를 시작하는 데 필요한 에너지를 얻을 수 있습니다.

### 4. 일일 목표 설정 및 계획 세우기

일일 목표 설정 및 계획 세우기는 아침에 하는 중요한 습관입니다. 이를 통해 우리는 하루의 목표를 설정하고, 이를 달성하는 데 필요한 계획을 세울 수 있습니다. 이를 통해 우리는 하루를 시작하는 데 필요한 동기를 부여할 수 있습니다.

### 5. 긍정적인 마인드셋을 유지하기 위한 방법

긍정적인 마인드셋은 우리의 삶을 긍정적으로 변화하는 데 중요한 방법입니다. 아침에 긍정적인 마인드셋을 유지하는 것은 하루를 시작하는 데 필요한 에너지를 얻을 수 있습니다. 이를 통해 우리는 하루를 시작하는 데 필요한 에너지를 얻을 수 있습니다.

**III. 요약 및 다음 단계**
-------------------------

최고의 하루를 보내기 위한 5가지 아침습관을 요약하면, 아침에 일어나는 것부터 시작하여 건강한 아침식사, 명상 또는 운동, 일일 목표 설정 및 계획 세우기, 긍정적인 마인드셋을 유지하는 것입니다. 이러한 아침습관을 실천하여 우리는 최고의 하루를 보낼 수 있습니다.

이러한 아침습관을 실천하려면 일단 아침에 일어나는 것을 시작하세요. 그리고 건강한 아침식사를 하며, 명상 또는 운동을 하세요. 그리고 일일 목표를 설정하고, 긍정적인 마인드셋을 유지하세요. 이러한 아침습관을 실천하여 우리는 최고의 하루를 보낼 수 있습니다.

In [ ]:
topic = "30대에 꼭 해야할 10가지 일"
result = crew.kickoff(inputs={"topic": topic})

In [ ]:
Markdown(result)

##  다른 인기 있는 모델: LLM을 위한 에이전트들

#### Hugging Face (HuggingFaceHub endpoint)


In [ ]:

from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    huggingfacehub_api_token="<HF_TOKEN_HERE>",
    task="text-generation",
)

#### Mistral API

In [ ]:

OPENAI_API_KEY=your-mistral-api-key
OPENAI_API_BASE=https://api.mistral.ai/v1
OPENAI_MODEL_NAME="mistral-small"






#### Cohere



In [ ]:
from langchain_community.chat_models import ChatCohere
# Initialize language model
os.environ["COHERE_API_KEY"] = "your-cohere-api-key"
llm = ChatCohere()

로컬에서 Ollama 및 기타 Llama와 함께 Llama를 사용하려면 [모든 LLM에 연결하는 방법](https://docs.crewai.com/how-to/LLM-Connections/)에 대한 crewAI 문서를 확인해보세요.

In [ ]:
os.environ["OPENAI_API_KEY"] = "NA"

llm = ChatOpenAI(
    model = "crewai-llama2",
    base_url = "http://localhost:11434/v1")